## Exercise 4

In [ ]:
import torch
from Fast_MRI_dataloader import create_dataloaders
from tqdm import tqdm 
import matplotlib.pyplot as plt
from torch.fft import fft2, fftshift, ifft2
import numpy as np

Exericse 4b

In [ ]:
def get_k_space(input) :
    # get the k-space
    k_space = fftshift(fft2(input))
    return k_space

def get_partial_k_space(input,M) :
    return  torch.mul(input, M)

def get_accelerate_MRI(input) :
    return ifft2(input)

def soft_threshold(input, threshold) :
    idx = torch.abs(input) > threshold
    input[idx] = input[idx] * (torch.abs(input[idx]) - threshold)/torch.abs(input[idx])
    input[~idx] = 0

    return input

def ISTA(mu, shrinkage, K, y, M) :
    
    H, W = y.shape[1:]

    # initialize 
    input_images = y
    x_k = np.zeros((H, W))

    image_list = []

    for idx, y in tqdm(enumerate(input_images)):
        # print("shape of y: ", y.shape)
        x = ifft2(y)
        for i in range(K):
            # gradient step, z = f1(x_k)
            z = np.abs(x - mu*M)) 
        
            # soft thresholding
            x_k = soft_threshold(z, shrinkage)

        # store the results
        image_list.append(x_k)
    
    # convert to tensor
    x_out = torch.from_numpy(np.array(image_list)).float()

    return x_out